# Bayes GLM

Using new BayesGLM() class, replicates the same behavior as R's [arm::bayesglm()](https://cran.r-project.org/web/packages/arm/arm.pdf)

In [ ]:
%pip install rpy2

In [9]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [10]:
from bayesglm.bayesglm import BayesGLM

import numpy as np
import pandas as pd
import rpy2.robjects as robjects
from scipy.special import expit

from statsmodels.genmod import families
from statsmodels.genmod.generalized_linear_model import GLM

## Generate Test Data Set

In [12]:
np.random.seed(12345)
n = 100
x1 = np.random.normal(size=n)
x2 = np.random.binomial(1, 0.5, n)
b0 = 1
b1 = 1.5
b2 = 2
const = np.ones(n)
y = np.random.binomial(1, expit(b0 + b1 * x1 + b2 * x2), n)
X = np.transpose(np.vstack([const, x1, x2]))

df = pd.DataFrame(
    np.hstack((y.reshape(100, 1), X.reshape(100, 3))),
    columns=["y", "const", "x1", "x2"],
)

df.head()

,y,const,x1,x2
0,1.0,1.0,-0.204708,0.0
1,1.0,1.0,0.478943,0.0
2,0.0,1.0,-0.519439,0.0
3,0.0,1.0,-0.555730,0.0
4,1.0,1.0,1.965781,0.0


In [ ]:
%%R
install.packages(c("arm"))

Pass the df generated in Python over to R, extract columns to vectors

In [14]:
%%R -i df

n <- 100
x1 <- df$x1
x2 <- df$x2
y <- df$y

head(df)

  y const         x1 x2
0 1     1 -0.2047077  0
1 1     1  0.4789433  0
2 0     1 -0.5194387  0
3 0     1 -0.5557303  0
4 1     1  1.9657806  0
5 1     1  1.3934058  0


Create models in R from the data generated in Python. 

## Recreate Models in R and Python

The [R `arm` documentation](https://cran.r-project.org/web/packages/arm/arm.pdf) includes a few model examples that are replicated here. Where the docs compare the `arm` output to the `glm` output, this notebook compares R's implementation to the proposed Pythonic implementation where possible. Note that where the R implementation calls `arm::bayesglm`, the Pythonic implementation will call `BayesGLM(...).fit()`

### Standard GLMs

In R build a standard GLM, compare to a standard GLM implementation in `statsmodels`. Compare outputs between R implementation and Python implementation throughout thi example notebook. 

In [15]:
%%R
M1 <- glm (y ~ x1 + x2, family=binomial(link="logit"))
arm::display (M1)

glm(formula = y ~ x1 + x2, family = binomial(link = "logit"))
            coef.est coef.se
(Intercept) 1.20     0.36   
x1          1.30     0.36   
x2          2.12     0.76   
---
  n = 100, k = 3
  residual deviance = 69.4, null deviance = 94.3 (difference = 24.9)


In [16]:
M1 = GLM(endog=df.y, exog=df.drop(columns="y"), family=families.Binomial()).fit()
M1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -34.684
Date:                Wed, 12 Jun 2024   Deviance:                       69.367
Time:                        07:10:24   Pearson chi2:                     76.7
No. Iterations:                     6   Pseudo R-squ. (CS):             0.2205
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2002      0.365      3.289      0.001       0.485       1.915
x1             1.2958      0.363      3.566      0.000       0.584       2.008
x2             2.1160      0.758      2.790      0.005       0.630       3.602
==============================================================================
"""

### Standard GLMs with infinite priors

In [17]:
%%R
M2 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=Inf, prior.df=Inf)
arm::display (M2) # just a test: this should be identical to classical logit

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = Inf, prior.df = Inf)
            coef.est coef.se
(Intercept) 1.20     0.36   
x1          1.29     0.36   
x2          2.11     0.76   
---
n = 100, k = 3
residual deviance = 69.4, null deviance = 94.3 (difference = 24.9)


In [18]:
M2 = BayesGLM(endog=df.y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=np.inf, prior_df=np.inf
)
M2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -34.684
Date:                Wed, 12 Jun 2024   Deviance:                       69.367
Time:                        07:10:24   Pearson chi2:                     76.6
No. Iterations:                     6   Pseudo R-squ. (CS):             0.2205
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1982      0.364      3.288      0.001       0.484       1.913
x1             1.2941      0.363      3.566      0.000       0.583       2.005
x2             2.1127      0.757      2.790      0.005       0.629       3.597
==============================================================================
"""

### BGLM default cauchy prior scale 2.5

In [19]:
%%R
M3 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"))
# default Cauchy prior with scale 2.5
arm::display (M3)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"))
            coef.est coef.se
(Intercept) 1.20     0.35   
x1          1.17     0.33   
x2          1.85     0.68   
---
n = 100, k = 3
residual deviance = 69.6, null deviance = 94.3 (difference = 24.7)


In [20]:
M3 = BayesGLM(endog=df.y, exog=df.drop(columns="y"), family=families.Binomial()).fit()
M3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -34.783
Date:                Wed, 12 Jun 2024   Deviance:                       69.566
Time:                        07:10:24   Pearson chi2:                     72.5
No. Iterations:                    12   Pseudo R-squ. (CS):             0.2190
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2029      0.354      3.401      0.001       0.510       1.896
x1             1.1667      0.327      3.564      0.000       0.525       1.808
x2             1.8542      0.678      2.737      0.006       0.526       3.182
==============================================================================
"""

### BGLM explicitly specify Cauchy prior, scale 2.5

In [21]:
%%R
M4 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=2.5, prior.df=1)
# Same as M3, explicitly specifying Cauchy prior with scale 2.5
arm::display (M4)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.df = 1)
            coef.est coef.se
(Intercept) 1.20     0.35   
x1          1.17     0.33   
x2          1.85     0.68   
---
n = 100, k = 3
residual deviance = 69.6, null deviance = 94.3 (difference = 24.7)


In [22]:
M4 = BayesGLM(endog=df.y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=2.5, prior_df=1
)
M4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -34.783
Date:                Wed, 12 Jun 2024   Deviance:                       69.566
Time:                        07:10:24   Pearson chi2:                     72.5
No. Iterations:                    12   Pseudo R-squ. (CS):             0.2190
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2029      0.354      3.401      0.001       0.510       1.896
x1             1.1667      0.327      3.564      0.000       0.525       1.808
x2             1.8542      0.678      2.737      0.006       0.526       3.182
==============================================================================
"""

### BGLM explicitly specify Cauchy prior, scale 2.5 and t7

In [23]:
%%R
M5 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=2.5, prior.df=7) # t_7 prior with scale 2.5
arm::display (M5)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.df = 7)
            coef.est coef.se
(Intercept) 1.20     0.35   
x1          1.17     0.33   
x2          1.88     0.69   


---
n = 100, k = 3
residual deviance = 69.5, null deviance = 94.3 (difference = 24.7)


In [24]:
M5 = BayesGLM(endog=df.y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=2.5, prior_df=7
)
M5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -34.772
Date:                Wed, 12 Jun 2024   Deviance:                       69.544
Time:                        07:10:24   Pearson chi2:                     72.8
No. Iterations:                     9   Pseudo R-squ. (CS):             0.2191
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1977      0.354      3.385      0.001       0.504       1.891
x1             1.1692      0.328      3.565      0.000       0.526       1.812
x2             1.8796      0.686      2.742      0.006       0.536       3.223
==============================================================================
"""

### BGLM explicitly specify normal prior with scale 2.5

In [25]:
%%R
M6 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=2.5, prior.df=Inf) # normal prior with scale 2.5
arm::display (M6)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.df = Inf)
            coef.est coef.se
(Intercept) 1.20     0.35   
x1          1.17     0.33   
x2          1.89     0.69   
---
n = 100, k = 3
residual deviance = 69.5, null deviance = 94.3 (difference = 24.7)


In [26]:
M6 = BayesGLM(endog=df.y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=2.5, prior_df=np.inf
)
M6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -34.768
Date:                Wed, 12 Jun 2024   Deviance:                       69.535
Time:                        07:10:25   Pearson chi2:                     73.0
No. Iterations:                     6   Pseudo R-squ. (CS):             0.2192
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.1980      0.354      3.383      0.001       0.504       1.892
x1             1.1710      0.328      3.566      0.000       0.527       1.815
x2             1.8884      0.688      2.743      0.006       0.539       3.237
==============================================================================
"""

### Create Separation

In [27]:
%%R
# augment y as shown in the docs

# Create separation: set y=1 whenever x2=1
# Now it should blow up without the prior!
y <- ifelse (x2==1, 1, 0)
y

  [1] 0 0 0 0 0 0 1 1 1 0 0 1 1 0 0 1 0 0 0 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 0 0 0
 [38] 0 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 0 0 1 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 1
 [75] 1 0 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 1 0 0 0 0 0 1 0 0


In [28]:
# import the augmented y into Python
y = np.array(robjects.globalenv["y"])
y

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.])

In [29]:
%%R
M1 <- glm (y ~ x1 + x2, family=binomial(link="logit"))
arm::display (M1)

glm(formula = y ~ x1 + x2, family = binomial(link = "logit"))
            coef.est coef.se 
(Intercept)   -26.57 47437.64
x1              0.00 34673.34
x2             53.13 72493.65
---
  n = 100, k = 3
  residual deviance = 0.0, null deviance = 136.7 (difference = 136.7)


In [30]:
M1 = GLM(endog=y, exog=df.drop(columns="y"), family=families.Binomial()).fit()
M1.summary()

/home/chris/Documents/Github/bayesglm/.venv/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/chris/Documents/Github/bayesglm/.venv/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/chris/Documents/Github/bayesglm/.venv/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/chris/Documents/Github/bayesglm/.venv/lib/python3.8/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Per

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -2.1433e-09
Date:                Wed, 12 Jun 2024   Deviance:                   4.2867e-09
Time:                        07:10:25   Pearson chi2:                 2.14e-09
No. Iterations:                    23   Pseudo R-squ. (CS):             0.7450
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -24.5661   1.75e+04     -0.001      0.999   -3.42e+04    3.42e+04
x1          9.104e-15   1.28e+04   7.14e-19      1.000    -2.5e+04     2.5e+04
x2            49.1321   2.67e+04      0.002      0.999   -5.22e+04    5.23e+04
==============================================================================
"""

### Same behavior for M2 and M2

In [31]:
%%R
M2 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=Inf, prior.scale.for.intercept=Inf) # Same as M1
arm::display (M2)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = Inf, prior.scale.for.intercept = Inf)
            coef.est coef.se 
(Intercept)   -26.57 47437.64
x1              0.00 34673.34
x2             53.13 72493.65
---
n = 100, k = 3
residual deviance = 0.0, null deviance = 136.7 (difference = 136.7)


In [32]:
# This will break with no priors! As there's perfect separation.
try:
    M2 = BayesGLM(endog=y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
        prior_scale=np.inf, prior_df=np.inf
    )
    M2.summary()
except Exception:
    print("Expected failure due to perfect separation")

Expected failure due to perfect separation


### Same behavior for M3 and M3

In [33]:
%%R
M3 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"))
arm::display (M3)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"))
            coef.est coef.se
(Intercept) -5.86     2.37  
x1          -0.03     0.91  
x2          11.53     3.33  
---
n = 100, k = 3
residual deviance = 0.6, null deviance = 136.7 (difference = 136.0)


In [34]:
M3 = BayesGLM(endog=y, exog=df.drop(columns="y"), family=families.Binomial()).fit()
M3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:               -0.30914
Date:                Wed, 12 Jun 2024   Deviance:                      0.61828
Time:                        07:10:25   Pearson chi2:                    0.310
No. Iterations:                    25   Pseudo R-squ. (CS):             0.7435
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.8636      2.374     -2.470      0.014     -10.516      -1.211
x1            -0.0333      0.914     -0.036      0.971      -1.825       1.759
x2            11.5296      3.334      3.458      0.001       4.994      18.065
==============================================================================
"""

### Same behavior for M4 and M4

In [35]:
%%R
M4 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=2.5, prior.scale.for.intercept=10) # Same as M3
arm::display (M4)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.scale.for.intercept = 10)
            coef.est coef.se
(Intercept) -5.86     2.37  
x1          -0.03     0.91  
x2          11.53     3.33  
---
n = 100, k = 3
residual deviance = 0.6, null deviance = 136.7 (difference = 136.0)


In [36]:
M4 = BayesGLM(endog=y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=2.5, prior_scale_for_intercept=10
)
M4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:               -0.30914
Date:                Wed, 12 Jun 2024   Deviance:                      0.61828
Time:                        07:10:25   Pearson chi2:                    0.310
No. Iterations:                    25   Pseudo R-squ. (CS):             0.7435
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.8636      2.374     -2.470      0.014     -10.516      -1.211
x1            -0.0333      0.914     -0.036      0.971      -1.825       1.759
x2            11.5296      3.334      3.458      0.001       4.994      18.065
==============================================================================
"""

### Same behavior for M5

In [37]:
%%R
M5 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=2.5, prior.df=7)
arm::display (M5)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.df = 7)
            coef.est coef.se
(Intercept) -4.58     1.26  
x1          -0.08     0.72  
x2           8.90     1.71  
---
n = 100, k = 3
residual deviance = 2.3, null deviance = 136.7 (difference = 134.4)


In [38]:
M5 = BayesGLM(endog=y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=2.5, prior_df=7
)
M5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -1.1395
Date:                Wed, 12 Jun 2024   Deviance:                       2.2790
Time:                        07:10:25   Pearson chi2:                     1.15
No. Iterations:                    20   Pseudo R-squ. (CS):             0.7392
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.5828      1.260     -3.636      0.000      -7.053      -2.112
x1            -0.0769      0.724     -0.106      0.915      -1.496       1.343
x2             8.9039      1.707      5.215      0.000       5.557      12.250
==============================================================================
"""

### Same behavior with M6

In [39]:
%%R
M6 <- arm::bayesglm (y ~ x1 + x2, family=binomial(link="logit"),
prior.scale=2.5, prior.df=Inf)
arm::display (M6)

arm::bayesglm(formula = y ~ x1 + x2, family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.df = Inf)
            coef.est coef.se
(Intercept) -3.84     0.87  
x1          -0.10     0.57  
x2           7.40     1.16  
---
n = 100, k = 3
residual deviance = 4.8, null deviance = 136.7 (difference = 131.9)


In [40]:
M6 = BayesGLM(endog=y, exog=df.drop(columns="y"), family=families.Binomial()).fit(
    prior_scale=2.5, prior_df=np.inf
)
M6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Binomial   Df Model:                            2
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -2.3985
Date:                Wed, 12 Jun 2024   Deviance:                       4.7969
Time:                        07:10:25   Pearson chi2:                     2.43
No. Iterations:                     7   Pseudo R-squ. (CS):             0.7325
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.8411      0.875     -4.391      0.000      -5.556      -2.127
x1            -0.0999      0.571     -0.175      0.861      -1.219       1.019
x2             7.4005      1.165      6.355      0.000       5.118       9.683
==============================================================================
"""

### bayesglm with gaussian family (bayes lm)

In [41]:
%%R
# data setup
sigma <- 5
b0 <- 1
b1 <- 1.5
b2 <- 2
y2 <- rnorm (n, b0+b1*x1+b2*x2, sigma)

In [42]:
y2 = np.array(robjects.globalenv["y2"])

### Compare behavior M7

In [43]:
%%R
# note that default family is Gaussian
M7 <- arm::bayesglm (y2 ~ x1 + x2, prior.scale=Inf, prior.df=Inf)
arm::display (M7)

arm::bayesglm(formula = y2 ~ x1 + x2, prior.scale = Inf, prior.df = Inf)
            coef.est coef.se
(Intercept) 0.28     0.68   
x1          2.02     0.49   
x2          3.78     1.03   
---
n = 100, k = 3
residual deviance = 2493.2, null deviance = 3183.3 (difference = 690.1)
overdispersion parameter = 25.7
residual sd is sqrt(overdispersion) = 5.07


In [44]:
M7 = BayesGLM(endog=y2, exog=df.drop(columns="y"), family=families.Gaussian()).fit(
    prior_scale=np.inf, prior_df=np.inf
)
M7.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       97
Model Family:                Gaussian   Df Model:                            2
Link Function:               Identity   Scale:                          25.703
Method:                         BIRLS   Log-Likelihood:                -302.70
Date:                Wed, 12 Jun 2024   Deviance:                       2493.2
Time:                        07:10:26   Pearson chi2:                 2.49e+03
No. Iterations:                     3   Pseudo R-squ. (CS):             0.2358
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2753      0.675      0.408      0.684      -1.048       1.599
x1             2.0201      0.494      4.093      0.000       1.053       2.988
x2             3.7787      1.032      3.661      0.000       1.756       5.801
==============================================================================
"""

### bayesglm with categorical variables

In [45]:
%%R
z1 <- trunc(runif(n, 4, 9))
levels(factor(z1))
z2 <- trunc(runif(n, 15, 19))
levels(factor(z2))

[1] "15" "16" "17" "18"


In [46]:
z1 = np.array(robjects.globalenv["z1"])
z2 = np.array(robjects.globalenv["z2"])

### Compare behavior M8

In [47]:
%%R
## drop the base level (R default)
M8 <- arm::bayesglm (y ~ x1 + factor(z1) + factor(z2),
family=binomial(link="logit"), prior.scale=2.5, prior.df=Inf)
arm::display (M8)

arm::bayesglm(formula = y ~ x1 + factor(z1) + factor(z2), family = binomial(link = "logit"), 
    prior.scale = 2.5, prior.df = Inf)
             coef.est coef.se
(Intercept)  -0.77     0.51  
x1           -0.34     0.21  
factor(z1)5   0.71     0.61  
factor(z1)6   0.66     0.65  
factor(z1)7   0.22     0.63  
factor(z1)8   0.93     0.69  
factor(z2)16  0.12     0.62  
factor(z2)17 -0.18     0.55  
factor(z2)18  0.24     0.59  
---
n = 100, k = 9
residual deviance = 131.7, null deviance = 136.7 (difference = 5.0)


In [48]:
df2 = pd.concat(
    [
        pd.Series(const, name="const"),
        pd.Series(x1, name="x1"),
        pd.get_dummies(z1, drop_first=True),
        pd.get_dummies(z2, drop_first=True),
    ],
    axis=1,
).astype(float)
df2.head()

,const,x1,5.0,6.0,7.0,8.0,16.0,17.0,18.0
0,1.0,-0.204708,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,0.478943,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,-0.519439,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,1.0,-0.555730,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.965781,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [49]:
M8 = BayesGLM(endog=y, exog=df2, family=families.Binomial()).fit(
    prior_scale=2.5, prior_df=np.inf
)
M8.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                       BayesGLM   Df Residuals:                       91
Model Family:                Binomial   Df Model:                            8
Link Function:                  Logit   Scale:                          1.0000
Method:                         BIRLS   Log-Likelihood:                -65.848
Date:                Wed, 12 Jun 2024   Deviance:                       131.70
Time:                        07:10:26   Pearson chi2:                     99.3
No. Iterations:                     5   Pseudo R-squ. (CS):            0.04846
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7731      0.510     -1.517      0.129      -1.772       0.226
x1            -0.3385      0.210     -1.609      0.108      -0.751       0.074
5.0            0.7052      0.605      1.166      0.244      -0.481       1.891
6.0            0.6554      0.653      1.004      0.316      -0.625       1.935
7.0            0.2210      0.635      0.348      0.728      -1.023       1.465
8.0            0.9332      0.685      1.362      0.173      -0.410       2.276
16.0           0.1201      0.625      0.192      0.848      -1.105       1.345
17.0          -0.1779      0.551     -0.323      0.747      -1.257       0.901
18.0           0.2414      0.590      0.409      0.682      -0.914       1.397
==============================================================================
"""

There are two more examples given in the documentation, but they're tedious to replicate within the limitation of rpy2 behavior.